In [1]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

# Statistical Test
* Alpha = 0.05 
* Formula: https://www.jmlr.org/papers/volume7/demsar06a/demsar06a.pdf
* Friedman Test Package: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.friedmanchisquare.html
* Friedman Test Tutorial: https://www.geeksforgeeks.org/friedman-test/
* Nemenyi Package: https://scikit-posthocs.readthedocs.io/en/latest/generated/scikit_posthocs.posthoc_nemenyi_friedman.html
* Nemenyi Tutorial: https://www.geeksforgeeks.org/how-to-perform-the-nemenyi-test-in-python/

In [2]:
# Read the experimenr results
df_class = pd.read_excel("../output/experiment/class_experiment_comparison_result.xlsx", header=[0, 1], index_col=0)
df_class = df_class.iloc[:-2]

df_association = pd.read_excel("../output/experiment/association_experiment_comparison_result.xlsx", header=[0, 1], index_col=0)
df_association = df_association.iloc[:-2]

In [3]:
# Set up the values to be compared
accuracy_selected_columns = [
    ('GPT-o1', 'F-0.5'),
    ('Llama3-8B', 'F-0.5'),
    ('Qwen-14B', 'F-0.5'),
    ('VN(Precision-Oriented)', 'F-0.5')
]
accuracy_optional_selected_columns = [
    ('GPT-o1(Opt)', 'F-0.5'),
    ('Llama3-8B(Opt)', 'F-0.5'),
    ('Qwen-14B(Opt)', 'F-0.5'),
    ('VN(Precision-Oriented)', 'F-0.5')
]

completeness_selected_columns = [
    ('GPT-o1', 'F-2'),
    ('Llama3-8B', 'F-2'),
    ('Qwen-14B', 'F-2'),
    ('VN(Recall-Oriented)', 'F-2')
]
completeness_optional_selected_columns = [
    ('GPT-o1(Opt)', 'F-2'),
    ('Llama3-8B(Opt)', 'F-2'),
    ('Qwen-14B(Opt)', 'F-2'),
    ('VN(Recall-Oriented)', 'F-2')
]

class_accuracy_df = df_class[accuracy_selected_columns]
class_accuracy_opt_df = df_class[accuracy_optional_selected_columns]
class_completeness_df = df_class[completeness_selected_columns]
class_completnesss_opt_df = df_class[completeness_optional_selected_columns]

association_accuracy_df = df_association[accuracy_selected_columns[:-1]]
association_accuracy_opt_df = df_association[accuracy_optional_selected_columns[:-1]]
association_completeness_df = df_association[completeness_selected_columns[:-1]]
association_completnesss_opt_df = df_association[completeness_optional_selected_columns[:-1]]

association_completnesss_opt_df

,GPT-o1(Opt),Llama3-8B(Opt),Qwen-14B(Opt)
,F-2,F-2,F-2
recycling,0.595,0.601,0.614
supermarket,0.902,0.690,0.706
planningpoker,0.871,0.720,0.779
camperplus,0.621,0.383,0.539
grocery,0.896,0.588,0.670
sports,0.870,0.582,0.624
ticket,0.965,0.505,0.710
school,0.682,0.342,0.485
fish&chips,0.896,0.577,0.734


In [4]:
def run_friedman_nemenyi_test(df, higher_is_better=True, alpha=0.05):
    """
    Run Friedman test and Nemenyi post-hoc test on a wide-format DataFrame.
    
    Parameters:
    - df (pd.DataFrame): Each column is a model, each row is a dataset/block.
    - higher_is_better (bool): If True, higher accuracy gets rank 1. If False, lower is better (e.g., error rate).
    - alpha (float): Significance level for the Friedman test.
    
    Returns:
    - statistic, p_value, posthoc_matrix (or None if not significant)
    """

    model_cols = df.columns.tolist()

    # Step 1: Rank the data (within each row)
    ranked_df = df.copy()
    ranked_df[model_cols] = df[model_cols].rank(
        method='average', axis=1, ascending=not higher_is_better
    )

    # Step 2: Friedman test
    ranks = [ranked_df[model].values.flatten() for model in model_cols]
    statistic, p_value = friedmanchisquare(*ranks)

    print("=== Friedman Test ===")
    print(f"Statistic: {statistic:.4f}")
    print(f"P-value:   {p_value:.4f}")

    # Step 3: Nemenyi post-hoc (if significant)
    if p_value < alpha:
        print("\n=== Post-hoc Nemenyi Test ===")
        data_matrix = ranked_df[model_cols].to_numpy()
        posthoc = sp.posthoc_nemenyi_friedman(data_matrix)
        posthoc.index = [model[0] for model in model_cols]
        posthoc.columns = [model[0] for model in model_cols]
        print(statistic)
        # print(posthoc)
        return statistic, p_value, posthoc
    else:
        print("\nNo significant difference found. Post-hoc test not needed.")
        return statistic, p_value, None

In [5]:
dfs = [class_accuracy_df, class_accuracy_opt_df, class_completeness_df, class_completnesss_opt_df,
       association_accuracy_df, association_accuracy_opt_df, association_completeness_df, association_completnesss_opt_df]
sheet_names = [
    "Class-Accuracy",
    "Class-Accuracy-Optional",
    "Class-Completeness",
    "Class-Completeness-Optional",
    "Ass-Accuracy",
    "Ass-Accuracy-Optional",
    "Ass-Completeness",
    "Ass-Completeness-Optional"
]

with pd.ExcelWriter("friedman_test_results.xlsx", engine='openpyxl') as writer:
    for df, name in zip(dfs, sheet_names):
        statistic, p_value, posthoc = run_friedman_nemenyi_test(df)

        df.to_excel(writer, sheet_name=name)

        stat_start = len(df) + 5  
        stat_df = pd.DataFrame({"Statistic": [statistic], "p-value": [p_value]})
        stat_df.to_excel(writer, sheet_name=name, startrow=stat_start, index=False)

        posthoc_start = stat_start + len(stat_df) + 2 
        if isinstance(posthoc, pd.DataFrame):
            posthoc.to_excel(writer, sheet_name=name, startrow=posthoc_start)


=== Friedman Test ===
Statistic: 17.1333
P-value:   0.0007

=== Post-hoc Nemenyi Test ===
17.133333333333326
=== Friedman Test ===
Statistic: 16.2000
P-value:   0.0010

=== Post-hoc Nemenyi Test ===
16.19999999999999
=== Friedman Test ===
Statistic: 8.6000
P-value:   0.0351

=== Post-hoc Nemenyi Test ===
8.599999999999994
=== Friedman Test ===
Statistic: 13.9333
P-value:   0.0030

=== Post-hoc Nemenyi Test ===
13.933333333333337
=== Friedman Test ===
Statistic: 13.5556
P-value:   0.0011

=== Post-hoc Nemenyi Test ===
13.555555555555543
=== Friedman Test ===
Statistic: 13.5556
P-value:   0.0011

=== Post-hoc Nemenyi Test ===
13.555555555555543
=== Friedman Test ===
Statistic: 12.6667
P-value:   0.0018

=== Post-hoc Nemenyi Test ===
12.666666666666657
=== Friedman Test ===
Statistic: 12.6667
P-value:   0.0018

=== Post-hoc Nemenyi Test ===
12.666666666666657


# Class - Sampling

In [9]:
def run_friedman_test(df):
    stat, p = friedmanchisquare(*[df[col] for col in df.columns])
    return stat, p

In [15]:
# Step 0: Read the VN report once
df_VN = pd.read_excel("../VN/VN_Report.xlsx", sheet_name="Result", header=[0, 1], index_col=0)
experiment_type = "class"
# Step 1–4: Process each dataset and write to separate Excel sheets
with pd.ExcelWriter(f"{experiment_type}_combined_results_all_datasets.xlsx", engine='openpyxl') as writer:
    for dataset in ['recycling', 'supermarket', 'planningpoker', 
                    'camperplus', 'grocery', 'sports',
                    'ticket', 'school', 'fish&chips']:
        for round in range(1, 4):
            df_all = pd.DataFrame()

            for model in ["GPT-o1", "Llama3-8B", "Qwen-14B"]:
                df = pd.read_excel(f"../output/{experiment_type}/{model}/{dataset}/experiment_results.xlsx")
                df = df[["F-0.5", 'F-2']]
                # if model == "GPT-o1":
                #     print(dataset)
                #     print(df)

                if model != "GPT-o1":
                    df = df.sample(n=5).reset_index(drop=True)
                else:
                    df = df.reset_index(drop=True)

                df.columns = pd.MultiIndex.from_product([[model], df.columns])
                df_all = pd.concat([df_all, df], axis=1)

               
            if experiment_type.lower() == "class":
                # Extract VN metrics
                VN_precision_oriented = df_VN.loc[dataset]["VN(Precision-Oriented)"][["F-0.5", "F-2"]]
                df_VN_precision = pd.DataFrame([VN_precision_oriented] * 5).reset_index(drop=True)
                df_VN_precision.columns = pd.MultiIndex.from_product([["VN(Precision-Oriented)"], df_VN_precision.columns])

                VN_recall_oriented = df_VN.loc[dataset]["VN(Recall-Oriented)"][["F-0.5", "F-2"]]
                df_VN_recall = pd.DataFrame([VN_recall_oriented] * 5).reset_index(drop=True)
                df_VN_recall.columns = pd.MultiIndex.from_product([["VN(Recall-Oriented)"], df_VN_recall.columns])

                # Combine VN and model results
                df_VN_combined = pd.concat([df_VN_precision, df_VN_recall], axis=1)
                df_final = pd.concat([df_all, df_VN_combined], axis=1)
                test_df_accuracy = df_final[accuracy_selected_columns]
                test_df_completeness = df_final[completeness_selected_columns]
            else:
                df_final = df_all
                test_df_accuracy = df_final[accuracy_selected_columns[:-1]]
                test_df_completeness = df_final[completeness_selected_columns[:-1]]

            # print(accuracy_selected_columns[:-1])
            stat_acc, p_acc = run_friedman_test(test_df_accuracy)
            stat_comp, p_comp = run_friedman_test(test_df_completeness)
            
            
            df_stats = pd.DataFrame({
                "Metric": ["Accuracy", "Completeness"],
                "Friedman χ²": [stat_acc, stat_comp],
                "p-value": [p_acc, p_comp]
            })
            posthoc_acc = None
            posthoc_comp = None
            if p_acc < 0.05:
                # print("Do posthoc for acc")
                posthoc_acc = sp.posthoc_nemenyi_friedman(test_df_accuracy.values)
                # Assign row and column labels for readability
                posthoc_acc.columns = test_df_accuracy.columns
                posthoc_acc.index = test_df_accuracy.columns
                # print(posthoc_acc)
            if p_comp < 0.05:
                # print("Do posthoc for comp")
                posthoc_comp = sp.posthoc_nemenyi_friedman(test_df_completeness.values)
                # Assign row and column labels for readability
                posthoc_comp.columns = test_df_completeness.columns
                posthoc_comp.index = test_df_completeness.columns
            main_table_height = len(df_final) + 5  # +1 for column header

            # Write to Excel sheet
            sheet_name = f"{dataset}_{round}"
            df_final.to_excel(writer, sheet_name=sheet_name)
            df_stats.to_excel(writer, sheet_name=sheet_name, startrow=main_table_height, index=False)

            posthoc_acc_start_row = main_table_height + len(df_stats) + 5

            worksheet = writer.sheets[sheet_name]

            if posthoc_acc is not None:
                # Add label
                worksheet.cell(row=posthoc_acc_start_row, column=1, value="Nemenyi: Compare Accuracy")
                # Write table
                posthoc_acc.to_excel(writer, sheet_name=sheet_name, startrow=posthoc_acc_start_row + 1)

                if posthoc_comp is not None:
                    # Add label to the right of accuracy block
                    posthoc_comp_start_col = posthoc_acc.shape[1] + 4
                    worksheet.cell(row=posthoc_acc_start_row, column=posthoc_comp_start_col + 1, value="Nemenyi: Compare Completeness")
                    posthoc_comp.to_excel(
                        writer,
                        sheet_name=sheet_name,
                        startrow=posthoc_acc_start_row + 1,
                        startcol=posthoc_comp_start_col
                    )

            elif posthoc_comp is not None:
                # Only completeness present
                worksheet.cell(row=posthoc_acc_start_row, column=1, value="Compare Completeness")
                posthoc_comp.to_excel(writer, sheet_name=sheet_name, startrow=posthoc_acc_start_row + 1)


In [11]:
from scipy.stats import friedmanchisquare
 
gpt_scores = [0.873, 0.821, 0.933, 0.8, 0.847]       # 5 trials
llama_scores = [0.392, 0.509, 0.522, 0.633, 0.513]      # 5 trials (downsampled)
qwen_scores = [0.67, 0.757, 0.684, 0.6, 0.69]      # 5 trials (copied for each trial)
vn_scores = [0.692, 0.692, 0.692, 0.692, 0.692]
 
stat, p = friedmanchisquare(gpt_scores, llama_scores, qwen_scores, vn_scores)
print(f"Friedman χ² = {stat:.3f}, p = {p:.6f}")

Friedman χ² = 12.840, p = 0.004996
